In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

In [2]:

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_student_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_student_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
#make sure columns name
school_student_complete.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
#create array of unique school names
unique_school_names = school_student_complete['School ID'].unique()
unique_school_names

#gives the length of unique school names to give us how many schools
school_count = len(unique_school_names)

school_count



15

In [5]:
#create array of unique student names
unique_students_names = school_student_complete['Student ID'].unique()
#gives the length of unique student names to give us how many student
students_count = len(unique_students_names)
students_count


39170

In [6]:
#total budget
total_budget = school_student_complete['budget'].sum()
total_budget

82932329558

In [7]:
#average math score calculation
avg_math_score = school_student_complete['math_score'].mean()
avg_math_score

78.98537145774827

In [8]:
#average reading score calculation
avg_reading_score =school_student_complete['reading_score'].mean()
avg_reading_score

81.87784018381414

In [9]:
#calculations for number and % passing math using loc[] or .apply(lambda x: x[x>=70].count())
num_passing_math = school_student_complete.loc[school_student_complete['math_score'] >= 70]['math_score'].count()

perc_pass_math = num_passing_math/students_count 

perc_pass_math

0.749808526933878

In [10]:
#calculations for number and % passing reading
num_passing_reading = school_student_complete.loc[school_student_complete['reading_score'] >= 70]['reading_score'].count()
perc_pass_reading = num_passing_reading/students_count 

perc_pass_reading

0.8580546336482001

In [11]:
#Overall Passing Rate Calculations
overall_pass = (avg_math_score+avg_reading_score)/2
overall_pass

80.43160582078121

In [12]:
#district dataframe from dictionary
 
district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [students_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading":[perc_pass_reading],
    "% Passing Math": [perc_pass_math],
    "Overall Passing Rate": [overall_pass]

})
district_summary

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,39170,82932329558,81.87784,78.985371,0.858055,0.749809,80.431606


In [13]:
#format cells
dist_sum = district_summary.style.format({"Total Budget": "$ {:,.2f}", 
                       "Average Reading Score": "{:.2f}", 
                       "Average Math Score": "{:.1f}", 
                       "% Passing Math": "{:.2%}", 
                       "% Passing Reading": "{:.2%}", 
                       "Overall Passing Rate": "{:.2%}"})
dist_sum

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,39170,"$ 82,932,329,558.00",81.88,79.0,85.81%,74.98%,8043.16%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [14]:
#groups by school

#by_school = school_student_complete.set_index=('school_name').groupby(['school_name'])
#assignee_work_df = pd.DataFrame(by_school["type"])
#assignee_work_df.head()


In [16]:
#groups by school
grouped_school_name = school_student_complete.groupby('school_name')

#calculate school type,budget,student per budget, avg math score , avg reading score
school_type    = grouped_school_name['type'].first()

#Total student
stu_count      = grouped_school_name['Student ID'].count()

school_budget  = grouped_school_name['budget'].first()
per_stu_budget = school_budget/stu_count
avg_math_score = grouped_school_name['math_score'].mean()
avg_reading_score = grouped_school_name['reading_score'].mean()

pass_math_scores = grouped_school_name['math_score'].apply(lambda x: x[x>=70].count())
pass_math_percentage = (pass_math_scores/stu_count)*100

pass_read_scores = grouped_school_name['reading_score'].apply(lambda x: x[x>=70].count())
pass_read_percentage = (pass_read_scores/stu_count)*100

In [23]:
#ctreat dataframe
df_by_school_name = pd.DataFrame({'School Type':school_type, 'Total Students':stu_count,  'Total School Budget':school_budget, 'Per Student Budget':per_stu_budget, 'Average Math Score':avg_math_score, 'Average Reading Score':avg_reading_score, '% Passing Math':pass_math_percentage, '% Passing Reading':pass_read_percentage})
df_by_school_name['% Overall Passing Rate'] = (df_by_school_name['% Passing Math']+df_by_school_name['% Passing Reading'])/2
top_performing_schools = df_by_school_name.copy()

#Formatting dataframe
top_performing_schools["Total School Budget"] = top_performing_schools["Total School Budget"].map("${:,.2f}".format)
top_performing_schools["Per Student Budget"] = top_performing_schools["Per Student Budget"].map("${:,.2f}".format)
top_performing_schools["% Overall Passing Rate"] = top_performing_schools["% Overall Passing Rate"].map("%{:,.2f}".format)
top_performing_schools["% Passing Reading"] = top_performing_schools["% Passing Reading"].map("% {:,.2f}".format)
top_performing_schools["% Passing Math"] = top_performing_schools["% Passing Math"].map("%{:,.2f}".format)

#overall_pass_rate 
#df_by_school_name
top_performing_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,%66.68,% 81.93,%74.31
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,%94.13,% 97.04,%95.59
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,%65.99,% 80.74,%73.36
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,%68.31,% 79.30,%73.80
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,%93.39,% 97.14,%95.27
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,%66.75,% 80.86,%73.81
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,%92.51,% 96.25,%94.38
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,%65.68,% 81.32,%73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,%66.06,% 81.22,%73.64


In [ ]:
#groups by school
#by_school = school_data .groupby(['school_name'])
#groups by school
#by_school = school_data.groupby(['school_name'])
#by_school = school_data.set_index('school_name').groupby(['school_name'])
#by_school

In [ ]:
#grouby_school_df = pd.DataFrame(by_school)
#grouby_school_df

In [ ]:
#school types
#sch_types = pd.DataFrame(by_school)['type']
#sch_types
# Count how many of each component Assignees worked on and create DataFrame
#assignee_work_df = pd.DataFrame(assignee_group["Component"].value_counts())
#assignee_work_df.head()

In [ ]:
#sch_size = by_school.set_index('school_name')['size']
#sch_size

In [ ]:
# total students by school
#stu_per_sch = school_data_complete['Student ID'].count()

# school budget
#sch_budget = school_data_complete['budget'].count()

#per student budget
#stu_budget = school_data_complete ['budget'] /school_data_complete ['size']

#avg math and reading scores by school
#avg_math = school_data_complete['math_score'].mean()
#avg_read = school_data_complete['reading_score'].mean()

In [ ]:
# % passing scores
#pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 

In [ ]:

#pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 
#pass_read
#overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_per_sch 
#overall 

In [ ]:
#sch_summary = pd.DataFrame({
#"School Type": sch_types,
#"Total Students": stu_per_sch,
#"Per Student Budget": stu_budget,
#"Total School Budget": sch_budget,
#"Average Math Score": avg_math,
#"Average Reading Score": avg_read,
#"% Passing Math": pass_math,
#"% Passing Reading": pass_read,
#"Overall Passing Rate": overall
#})


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [26]:
# sort values by passing rate and then only print top 5 
#top_5 = sch_summary.sort_values("Overall Passing Rate", ascending = False)
#top_5.head().style.format({'Total Students': '{:,}',
 #                          "Total School Budget": "${:,}", 
  #                         "Per Student Budget": "${:.0f}", 
   #                        "% Passing Math": "{:.1%}", 
    #                       "% Passing Reading": "{:.1%}", 
     #                      "Overall Passing Rate": "{:.1%}"})
top5 = top_performing_schools.sort_values(by='% Overall Passing Rate',ascending=0).head(5)
top5

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,%94.13,% 97.04,%95.59
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,%93.27,% 97.31,%95.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,%93.39,% 97.14,%95.27
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,%94.59,% 95.95,%95.27
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,%93.87,% 96.54,%95.20


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [28]:
bttom5 = top_performing_schools.sort_values(by='% Overall Passing Rate',ascending= True).head(5)
bttom5

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,%66.37,% 80.22,%73.29
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,%65.99,% 80.74,%73.36
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,%65.68,% 81.32,%73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,%66.06,% 81.22,%73.64
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,%68.31,% 79.30,%73.80


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [29]:
#Create a pandas series for each grade. Hint: using a conditional statement.
grade_nine   = school_student_complete.loc[school_student_complete["grade"] == "9th", :]
grade_ten    = school_student_complete.loc[school_student_complete["grade"] == "10th", :]
grade_eleven = school_student_complete.loc[school_student_complete["grade"] == "11th", :]
grade_twelve = school_student_complete.loc[school_student_complete["grade"] == "12th", :]

#Group each series by school
math_series_nine = pd.Series(grade_nine.groupby(['school_name'])['math_score'].mean())
math_series_ten  = pd.Series(grade_ten.groupby(['school_name'])['math_score'].mean())
math_series_eleven = pd.Series(grade_eleven.groupby(['school_name'])['math_score'].mean())
math_series_twelve = pd.Series(grade_twelve.groupby(['school_name'])['math_score'].mean())

#Combine the series into a dataframe
math_score_grade = pd.DataFrame({"9th":math_series_nine,"10th":math_series_ten,"11th":math_series_eleven,"12th":math_series_twelve})

math_score_grade["9th"] = math_score_grade["9th"].map("{:,.2f}".format)
math_score_grade["10th"] = math_score_grade["10th"].map("{:,.2f}".format)
math_score_grade["11th"] = math_score_grade["11th"].map("{:,.2f}".format)
math_score_grade["12th"] = math_score_grade["12th"].map("{:,.2f}".format)


math_score_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [39]:
reading_series_nine = pd.Series(grade_nine.groupby(['school_name'])['reading_score'].mean())
reading_series_ten  = pd.Series(grade_ten.groupby(['school_name'])['reading_score'].mean())
reading_series_eleven = pd.Series(grade_eleven.groupby(['school_name'])['reading_score'].mean())
reading_series_twelve = pd.Series(grade_twelve.groupby(['school_name'])['reading_score'].mean())

reading_score_grade = pd.DataFrame({"9th":reading_series_nine,"10th":reading_series_ten,"11th":reading_series_eleven,"12th":reading_series_twelve})
#reading_score_grade

reading_score_grade["9th"] = reading_score_grade["9th"].map("{:,.2f}".format)
reading_score_grade["10th"] = reading_score_grade["10th"].map("{:,.2f}".format)
reading_score_grade["11th"] = reading_score_grade["11th"].map("{:,.2f}".format)
reading_score_grade["12th"] = reading_score_grade["12th"].map("{:,.2f}".format)

reading_score_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [32]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]


df_by_school_name["Spending Bins"] = pd.cut(df_by_school_name["Per Student Budget"],spending_bins,labels=group_names)
grouped_bins = df_by_school_name.groupby(["Spending Bins"])
column_mean = grouped_bins[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]].mean()
column_mean

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Bins,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [33]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
df_by_school_name["School_size"] = pd.cut(df_by_school_name["Total Students"],size_bins,labels=group_names)
grouped_size_bins = df_by_school_name.groupby(["School_size"])
column_means = grouped_size_bins[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]].mean()
column_means

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School_size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type

In [34]:
group_names = ["Charter","District"]
grouped_school_type = df_by_school_name.groupby(["School Type"])
mean_columns = grouped_school_type[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]].mean()
mean_columns

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
